In [1]:
import numpy as np
import pandas as pd

from ptplot import plotting, utilities

%reload_ext autoreload
%autoreload 2

We'll pick a single play to look at, just for demonstration purposes. I've somewhat-randomly chosen [this Cleveland Browns reverse + WR pass from the 2018 season](https://www.youtube.com/watch?v=Co5IaAFdaPU), which has player-tracking data available for it from Adam Sonty's [ngs_highlights repo](https://github.com/asonty/ngs_highlights) (specifically, the file used is located [here](https://raw.githubusercontent.com/asonty/ngs_highlights/20f74138f0a2f7e91879d31bbf2d8addc0bd59bc/play_data/2018_CLE_2018122305_1246.tsv)).

In [2]:
# Data file copied locally to ensure the demo works even if the ngs_highlights_repo is taken down
player_tracking_data = pd.read_csv(
    "2018_CLE_2018122305_1246.tsv",
    sep="\t"
)

In [3]:
player_tracking_data.head().T

,0,1,2,3,4
gameId,2018122305,2018122305,2018122305,2018122305,2018122305
playId,1246,1246,1246,1246,1246
playType,play_type_pass,play_type_pass,play_type_pass,play_type_pass,play_type_pass
season,2018,2018,2018,2018,2018
seasonType,REG,REG,REG,REG,REG
week,16,16,16,16,16
preSnapHomeScore,7,7,7,7,7
preSnapVisitorScore,0,0,0,0,0
playDirection,right,right,right,right,right
quarter,2,2,2,2,2


In [22]:
fig = plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], "x", "y",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    team_abbreviations="teamAbbr"
)
fig.show()

In [23]:
fig = plotting.plot_tracks(
    player_tracking_data, "x", "y", "displayName",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"].fillna(""), sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    team_abbreviations="teamAbbr"
)
fig.show()

In [24]:
fig = plotting.animate_play(
    player_tracking_data, "x", "y", "frame",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    #slider_label_generator=utilities.generate_time_elapsed_labels(player_tracking_data["time"].min(), "time"),
    team_abbreviations="teamAbbr",
    events_of_interest="event"
)
fig.show()

In [ ]:
post_script = """
console.log("{plot_id}");
var plot_div = document.getElementById("{plot_id}");
var main_svg = plot_div.querySelector(".main-svg");
main_svg.className += "wooooo";
"""
fig.write_html(
    "test.html", auto_play=False, include_plotlyjs="cdn", full_html=False, config={'displayModeBar': False},
    #post_script = post_script
)

In [27]:
fig = plotting.plot_positions(
    player_tracking_data[player_tracking_data["event"] == "pass_forward"], "y", "x",
    uniform_number="jerseyNumber",
    home_away_identifier="homeTeamFlag",
    hover_text=lambda data: data["displayName"].str.cat(data["position"], sep=" "),
    ball_identifier=lambda data: (data["displayName"] == "ball").values,
    team_abbreviations="teamAbbr",
    fig="nfl_vertical"
)
fig.update_layout(width=400, height=700)
fig.show()